# Modelo de prediccion de clima V2
Pronostica el clima desde la fecha seleccionada hasta 4 semanas después.

## Importar Librerías

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

## Cargar y Combinar los Datos

In [2]:
# Cargar los datos de cada municipio
departments = ['Suchitepéquez', 'Quetzaltenango', 'Escuintla']
municipalities = ['San Antonio Suchitepéquez', 'Mazatenango', 'Chicacao',
                  'Quetzaltenango', 'Cantel', 'Concepción Chiquirichapa',
                  'Escuintla', 'Santa Lucía Cotzumalguapa', 'Tiquisate']

all_data = []
for dep in departments:
    for mun in municipalities:
        if mun in ['San Antonio Suchitepéquez', 'Mazatenango', 'Chicacao'] and dep == 'Suchitepéquez':
            file_path = f"/content/drive/MyDrive/datos clima/{dep}_{mun}.csv"
        elif mun in ['Quetzaltenango', 'Cantel', 'Concepción Chiquirichapa'] and dep == 'Quetzaltenango':
            file_path = f"/content/drive/MyDrive/datos clima/{dep}_{mun}.csv"
        elif mun in ['Escuintla', 'Santa Lucía Cotzumalguapa', 'Tiquisate'] and dep == 'Escuintla':
            file_path = f"/content/drive/MyDrive/datos clima/{dep}_{mun}.csv"
        else:
            continue

        # Leer archivo CSV
        df = pd.read_csv(file_path)
        df['department'] = dep
        df['municipality'] = mun
        all_data.append(df)

# Combinar en un único DataFrame
data = pd.concat(all_data, ignore_index=True)

# Seleccionar columnas relevantes
data = data[['YEAR', 'DOY', 'T2M', 'RH2M', 'PRECTOTCORR', 'department', 'municipality']]

## Preprocesamiento y Generación de Ventanas

In [3]:
# Normalizar los datos
scaler = MinMaxScaler()
data[['T2M', 'RH2M', 'PRECTOTCORR']] = scaler.fit_transform(data[['T2M', 'RH2M', 'PRECTOTCORR']])

# Crear ventanas de secuencias
window_size = 7
X, y = [], []

for municipality in data['municipality'].unique():
    municipality_data = data[data['municipality'] == municipality]
    municipality_values = municipality_data[['T2M', 'RH2M', 'PRECTOTCORR']].values
    for i in range(len(municipality_values) - window_size):
        X.append(municipality_values[i:i+window_size])
        y.append(municipality_values[i+window_size])

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

<ipython-input-3-071548e93360>:16: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  X = torch.tensor(X, dtype=torch.float32)


## Definición del Modelo

In [4]:
class ClimatePredictor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ClimatePredictor, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        _, (hidden, _) = self.lstm(x)
        out = self.fc(hidden[-1])
        return out

# Hiperparámetros
input_size = 3  # T2M, RH2M, PRECTOTCORR
hidden_size = 64
output_size = 3  # Predicción de T2M, RH2M, PRECTOTCORR

model = ClimatePredictor(input_size, hidden_size, output_size)

## Entrenamiento del Modelo

In [10]:
# Configuración de entrenamiento
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 150

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    predictions = model(X_train)
    loss = criterion(predictions, y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

Epoch 10/150, Loss: 0.0048
Epoch 20/150, Loss: 0.0043
Epoch 30/150, Loss: 0.0041
Epoch 40/150, Loss: 0.0040
Epoch 50/150, Loss: 0.0039
Epoch 60/150, Loss: 0.0038
Epoch 70/150, Loss: 0.0038
Epoch 80/150, Loss: 0.0037
Epoch 90/150, Loss: 0.0037
Epoch 100/150, Loss: 0.0036
Epoch 110/150, Loss: 0.0036
Epoch 120/150, Loss: 0.0035
Epoch 130/150, Loss: 0.0035
Epoch 140/150, Loss: 0.0034
Epoch 150/150, Loss: 0.0034


## Evaluación del Modelo

In [11]:
model.eval()
with torch.no_grad():
    test_predictions = model(X_test)
    test_loss = criterion(test_predictions, y_test)
    print(f"Test Loss: {test_loss.item():.4f}")

Test Loss: 0.0033


## Predicción a Futuro

In [12]:
def predict_future(model, data, window_size, days):
    model.eval()
    predictions = []
    current_sequence = data[-window_size:]

    with torch.no_grad():
        for _ in range(days):
            current_sequence_tensor = torch.tensor(current_sequence, dtype=torch.float32).unsqueeze(0)
            prediction = model(current_sequence_tensor).squeeze(0).numpy()
            predictions.append(prediction)
            current_sequence = np.vstack((current_sequence[1:], prediction))

    return predictions

# Predicción para 28 días (4 semanas)
future_predictions = predict_future(model, data[['T2M', 'RH2M', 'PRECTOTCORR']].values, window_size, 28)

# Desnormalizar resultados para interpretarlos
future_predictions = scaler.inverse_transform(future_predictions)
print("Predicciones futuras:")
for i, pred in enumerate(future_predictions, 1):
    print(f"Day {i}: Temperature={pred[0]:.2f}, Humidity={pred[1]:.2f}, Rainfall={pred[2]:.2f}")

Predicciones futuras:
Day 1: Temperature=22.71, Humidity=86.55, Rainfall=8.86
Day 2: Temperature=22.55, Humidity=86.92, Rainfall=10.09
Day 3: Temperature=22.46, Humidity=86.89, Rainfall=10.74
Day 4: Temperature=22.37, Humidity=86.87, Rainfall=11.26
Day 5: Temperature=22.38, Humidity=86.64, Rainfall=11.61
Day 6: Temperature=22.43, Humidity=86.39, Rainfall=11.82
Day 7: Temperature=22.35, Humidity=86.36, Rainfall=11.92
Day 8: Temperature=22.17, Humidity=86.65, Rainfall=12.05
Day 9: Temperature=22.10, Humidity=86.68, Rainfall=12.13
Day 10: Temperature=22.04, Humidity=86.69, Rainfall=12.16
Day 11: Temperature=21.98, Humidity=86.70, Rainfall=12.17
Day 12: Temperature=21.92, Humidity=86.72, Rainfall=12.17
Day 13: Temperature=21.85, Humidity=86.76, Rainfall=12.15
Day 14: Temperature=21.77, Humidity=86.80, Rainfall=12.13
Day 15: Temperature=21.69, Humidity=86.85, Rainfall=12.10
Day 16: Temperature=21.62, Humidity=86.88, Rainfall=12.07
Day 17: Temperature=21.55, Humidity=86.91, Rainfall=12.04
Da

## Convertir el modelo a TorchScript (PyTorch Mobile)

In [9]:
import torch

scripted_model = torch.jit.script(model)
scripted_model.save("climate_predictor.pt")
